<a href="https://colab.research.google.com/github/enzocfb/general/blob/main/EDA_2023_deductivos_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [158]:
"""
Analisis de Deductivos para analizar causas
Impacto :

input:
      Consolidato total de Evaluados por UGS(Formato 06) para todos los años
        Consolidado que contiene los portafolios : 2024-02 , 2024-01, 2023-01 , 2022-02

      Deductivos reportados por UGT
      Fechas referenciales por grupos

output:
      Consolidado de evaluados, y su estado : "Deductivo", y "No Deductivo"

"""

'\nAnalisis de Deductivos para analizar causas\nImpacto :\n\ninput:\n      Consolidato total de Evaluados por UGS(Formato 06) para todos los años\n        Consolidado que contiene los portafolios : 2024-02 , 2024-01, 2023-01 , 2022-02\n\n      Deductivos reportados por UGT\n      Fechas referenciales por grupos\n\noutput:\n      Consolidado de evaluados, y su estado : "Deductivo", y "No Deductivo"\n\n'

In [159]:
import pandas as pd
import numpy as np


In [258]:
####################################################3
# Input
####################################################3

# Archivo completo de las evaluaciones realizadas por UGS, a partir de la cual se extrae la informacion del beneficiario,
# # Alcance : REGISTRADO- desde que se inicia en ESTADO REGISTRADO, LUEGO A ESTADO EVALUADO , luego  ESTADO VIABILIDAD y finalmente # a ESTADO DE DEDUCTIVIDAD
# KEY or INDICE del ARCHIVO:   CUV (Codigo que contiene el UIBGEO del CENTRO POBLADO + DNI del JEFE + AñO + NRO PORTAFOLIO)
#                              CONVENIO,  es una agrupacion de Beneficiarios (FALTA, no tiene en este momento esa data)

file = '/content/drive/MyDrive/@@@Dev/mvcs/Data/Formato06Completo.xlsx'

# 2. ARCHIVO DE DEDUCTIVOS (BENEFICIARIO QUE ESTA EN ESTADO = "DEDUCTIVO", es aquel que "SE EXCLUYE" DEL PROGRAMA. Este archivo es recibido de  UGT que contiene la informacion de todos aquellos que fueron
# ALCANCE : VIABLE-->DEDUCTIVO
# KEY or INDICE del ARCHIVO:
#                              CUV (Codigo que contiene el UIBGEO del CENTRO POBLADO + DNI del JEFE + AñO + NRO PORTAFOLIO) (SI TIENE)
#                              CONVENIO,  es una agrupacion de Beneficiarios,                                               (SI TIENE)

file_deductivos= '/content/drive/MyDrive/@@@Dev/mvcs/Data/LISTADO DE DEDUCTIVOS A (UGS).xlsx'


# Informacion a nivel de nucleos de las fechas de exp tecnico y fecha de Acta de inicio de Construccion
# Alcance : REGISTRADO-->EVALUADO-->VIABLE (TODOs: ESTADO_VIABLE = "VIABLE", INCLUIDO ESTADO_DEDUCTIVO = {"DEDUCTIVO", "NO DEDUCTIVO"] se une a nivel con archivo 2, con los
# 17 nov : Solo se tienen info para VIABLE-->DEDUCTIVO, falta a nivel de todos los VIABLES.
# KEY or INDICE del ARCHIVO:   CONVENIO,  es una agrupacion de Beneficiarios, (SI TIENE)

file_input_nucleo = '/content/drive/MyDrive/@@@Dev/mvcs/Data/NUCLEO - REPORTE COMPLEMENTARIO.xlsx'
file_input_2020_02 = '/content/drive/MyDrive/@@@Dev/mvcs/Data/Pnvr_formato06_20231122-15.57.07.xlsx'

file_base_total_consolidado = '/content/drive/MyDrive/@@@Dev/mvcs/Data/Base Total Usuarios del PNVR_v7_20.11.23.xlsx'
file_base_sample = '/content/drive/MyDrive/@@@Dev/mvcs/Data/Base sample.xlsx'



####################################################3
# Output
####################################################3
# output temporales
file_output_1 = '/content/drive/MyDrive/@@@Dev/mvcs/Data/CUVs no encontrados.xlsx'
file_output_2 = '/content/drive/MyDrive/@@@Dev/mvcs/Data/CUVs si encontrados.xlsx'


file_temp_1 = '/content/drive/MyDrive/@@@Dev/mvcs/Data/temp_1.xlsx'
file_temp_2 = '/content/drive/MyDrive/@@@Dev/mvcs/Data/temp_2.xlsx'
file_output_3 = '/content/drive/MyDrive/@@@Dev/mvcs/Data/temp_3.xlsx'
file_temp_4 = '/content/drive/MyDrive/@@@Dev/mvcs/Data/temp_4.xlsx'

# output
file_output_analisis = '/content/drive/MyDrive/@@@Dev/mvcs/Data/deductivos-completo-analysis.xlsx'

file_output_analisis_2 = '/content/drive/MyDrive/@@@Dev/mvcs/Data/deductivos-completo-analysis-2.xlsx'


# Constantes
constante_origen_PP068 = 'PP068'   # NUMERO DEL PROGRAMA ORIGEN

# Creamos ESTADOS : REGISTRADO, EVALUACION,  VIABILIDAD,
constante_estado_registrado = 'REGISTRADO'

constante_estado_evaluacion_evaluado = "EVALUADO"
constante_estado_evaluacion_rechaza = "RECHAZA_EVALUACION"
constante_estado_evaluacion_otros = 'OTROS_NO_EVALUADO'

constante_estado_viabilidad_viable = "VIABLE"
constante_estado_viabilidad_no_viable = "NO VIABLE"
constante_estado_sininfo = "SIN INFORMACION"
constante_estado_viabilidad_social_viable = 'SOCIALMENTE VIABLE'
constante_estado_viabilidad_social_no_viable = 'NO VIABLE SOCIALMENTE'

constante_estado_SI = "SI"
constante_estado_SI = "NO"


constante_estado_deductivo = "DEDUCTIVO"
constante_estado_no_deductivo = "NO_DEDUCTIVO"

# Mejoras
# Definir Estructura UGS
# Definir Estructura UGS + UGT




In [161]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [162]:
def convert_to_date(value):
    try:
        return pd.to_datetime(value)
    except ValueError:
        # Handle the case where parsing the date fails
        return value

# # Read the Excel file with the converters parameter
# df = pd.read_excel(file, converters={"Your_Column_Name": convert_to_date})

In [181]:
# La hoja excel, original se le transforma para poder ser leida
# Combinar celdas
# subir fila 6,
# bajamos los nombre de capos de la fila 5 a fila 6 (solo los que faltan)


df = pd.read_excel(file, converters={"CODIGO_CCPP": str, "UBIGEO": str})
# df = pd.read_excel(file, converters={"CODIGO_CCPP": str, "UBIGEO": str, "FEC_EVALUACION": convert_to_date})
df.head(3)

# CREACION DE ESTADOS PARA TABLA INICIAL



,NUMERO,ORIGEN,PORTAFOLIO,PORTAFOLIO_T,GRUPO,NRO ENCUESTA,CUV,CODIGO_CCPP,UBIGEO_temp,UBIGEO,...,CRITERIO_SOCIAL,CRITERIO_TECNICO,SOCIAL ESTADO DE LA FICHA,TÉCNICA,OBSERVACIONES,FECHA CADUCIDAD CSE,FECHA_CONSULTA_CSE,ACEPTA_ENTREVISTA,ESTADO_EVALUADO_T,FEC_EVALUACION
0,1,PP068,02-2024,2024-02,2,1,0808050259454185562402,0808050259,80805,080805,...,NaN,NaN,REGISTRADO,REGISTRADO,NaN,2028-08-19 00:00:00,2023-09-21 02:30:56,SI,SI,14-09-2023
1,2,PP068,02-2024,2024-02,2,2,0808050259000000002402,0808050259,80805,080805,...,NaN,NaN,REGISTRADO,SIN REGISTRO,NaN,NaN,NaN,Familia Ausente,NO,14-09-2023
2,3,PP068,02-2024,2024-02,2,3,0808050259475809342402,0808050259,80805,080805,...,NaN,NaN,REGISTRADO,REGISTRADO,NaN,2024-10-09 00:00:00,2023-09-21 02:37:17,SI,SI,14-09-2023


In [182]:
# Seleccionar campos requeridos para el analisis y construir las features

columns_selected_df = ['CUV','NUMERO', 'ORIGEN', 'PORTAFOLIO_T',  'CODIGO_CCPP', 'UBIGEO', 'DEPARTAMENTO','PROVINCIA', 'DISTRITO', 'CENTRO_POBLADO', 'COMUNIDAD',
                       	'PATERNO_JEFE_HOGAR', 'NOMBRES_JEFE_HOGAR', 'DNI_JEFE_HOGAR', 'SEXO_JEFE_HOGAR', 'EDAD_JEFE_HOGAR', 'FEC_NAC_JEFE_HOGAR',
												'CANT', 'CTD_DISCAPACIDAD', 'ALTURA', 'LATITUD', 'LONGITUD', 'CLASIFICACION_SOCIOECONOMICA_CSE', 'VIABILIDAD_TOTAL', 'VIABILIDAD_SOCIAL', 'VIABILIDAD_TECNICA','DNI_CYG','ACEPTA_ENTREVISTA', 'CRITERIO_SOCIAL', 'CRITERIO_TECNICO', 'FEC_EVALUACION', 'FECHA_CONSULTA_CSE']
df = df[columns_selected_df]

# renonmbar dni coyg por DNI_CYG

columns_selected_df_2020_02 = ['CODIGO CUV', 'DEPARTAMENTO', 'PROVINCIA', 'DISTRITO',
															'CENTRO POBLADO', 'COMUNIDAD', 'PATERNO', 'MATERNO', 'NOMBRES', 'DNI', 'SEXO',
                               'ALTURA','LATITUD', 'LONGITUD', 'DETERMINACIÓN DE LA VIABILIDAD', 'CRITERIO SOCIAL', 'CRITERIO TÉCNICO',
															'VIABILIDAD SOCIAL', 'VIABILIDAD TECNICA',
															'DNI_CYG',
															'CLASIFICACION SOCIOECONOMICA CSE'
															]



In [165]:
############################################################3
# Crear base unica, a partir de base consolidada y formato06
###############################################################################


In [166]:

# df = pd.read_excel(file)
# df = df[columns_selected_df]

# el campo CUV, tiene vacios, pero el campo DNI, no asi que lo elegimos como clave
# para ello verifricamos si este campo tiene duplicados




In [167]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26450 entries, 0 to 26449
Data columns (total 32 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   CUV                               26450 non-null  object 
 1   NUMERO                            26450 non-null  int64  
 2   ORIGEN                            26450 non-null  object 
 3   PORTAFOLIO_T                      26450 non-null  object 
 4   CODIGO_CCPP                       26450 non-null  object 
 5   UBIGEO                            26450 non-null  object 
 6   DEPARTAMENTO                      26450 non-null  object 
 7   PROVINCIA                         26450 non-null  object 
 8   DISTRITO                          26450 non-null  object 
 9   CENTRO_POBLADO                    26450 non-null  object 
 10  COMUNIDAD                         15300 non-null  object 
 11  PATERNO_JEFE_HOGAR                25454 non-null  object 
 12  NOMB

In [168]:
print(df.shape[0])
# Dropping rows where the 'Name' column is null
df_cleaned = df.dropna(subset=['CUV'], inplace = True)
print(df.shape[0])

26450
26450


In [169]:
df_base_consolidada = pd.read_excel(file_base_total_consolidado)
# df_base_consolidada = pd.read_excel(file_base_sample)

df_base_consolidada.head(3)

columns_selected_df_base = [ 'OBJECTID', 'CUV', 'Ubigeo CP', 'Ubigeo', 'Departamento', 'Provincia', 'Distrito', 'Centro Poblado', 'Comunidad', 'Ape. Paterno', 'Ape. Materno', 'Nombre', 'DNI', 'Sexo', 'edad', 'Fecha de Nacimiento',
                            'Número de Miembros', 'altitud', 'latitud', 'longitud']

new_columns_to_add = ['NUMERO', 'ORIGEN', 'PORTAFOLIO_T', 'CLASIFICACION_SOCIOECONOMICA_CSE', 'VIABILIDAD_TOTAL', 'VIABILIDAD_SOCIAL', 'VIABILIDAD_TECNICA',
                      'ACEPTA_ENTREVISTA', 'CRITERIO_SOCIAL', 'CRITERIO_TECNICO', 'FEC_EVALUACION', 'FECHA_CONSULTA_CSE']


# Filtramos los CUVs que no tienen

In [196]:
#===================================================================
# INICIAR DESDE AQUI
#===================================================================

df06 = df
df_base = df_base_consolidada[columns_selected_df_base]

print(df06.shape[0])
print(df_base.shape[0])


26450
75523


In [202]:
# Counting duplicated values in the 'Name' column
duplicates_count = df06.duplicated(subset='CUV').sum()
print("\nCount of duplicated values in the 'CUV' column:", duplicates_count)
# Count of duplicated values in the 'CUV' column: 5602

duplicates_count = df_base.duplicated(subset='CUV').sum()
print("\nCount of duplicated values in the 'CUV' column:", duplicates_count)
# Count of duplicated values in the 'CUV' column: 30083

# Se descarta dni como clave y solo se usa CUV, para ello elinamos los CUV que estan vacios




Count of duplicated values in the 'CUV' column: 5602

Count of duplicated values in the 'CUV' column: 25553


In [200]:

# Dropping rows with null values in the 'Name' column
df06 = df06.dropna(subset=['CUV'])
df_base = df_base.dropna(subset=['CUV'])



In [201]:
print(df06.shape[0])
print(df_base.shape[0])

26450
70992


In [203]:
# Eliminamos duplicados

df06 = df06.drop_duplicates(subset='CUV')
df_base = df_base.drop_duplicates(subset='CUV')

# Dropping duplicated values in the 'Name' column
# df_unique = df.drop_duplicates(subset='Name')

In [205]:
print(df06.shape[0])
print(df_base.shape[0])

20848
45439


In [133]:
# Converting the 'Age' column to integer data type
# df['CUV'] = df['CUV'].astype(int)
# df_base['CUV'] = df['CUV'].astype(int)

# df['CUV'] = np.int64(df['CUV'])
# df_base['CUV'] = np.int64(df_base['CUV'])

# # Setting the 'Name' column as the index

# df.set_index('CUV', inplace=True)
# df_base.set_index('CUV', inplace=True, drop=True)


In [135]:
  # # Add empty columns to the DataFrame
  # for col_name in new_columns_to_add:
  #     df_base[col_name] = None

In [206]:
# ================================================================
# renombar las columnas campos segun el df padre : df:
# ================================================================

df_base.rename(columns={'Ubigeo CP': 'CODIGO_CCPP'}, inplace=True)
df_base.rename(columns={'Ubigeo': 'UBIGEO'}, inplace=True)
df_base.rename(columns={'Departamento': 'DEPARTAMENTO'}, inplace=True)
df_base.rename(columns={'Provincia': 'PROVINCIA'}, inplace=True)
df_base.rename(columns={'Distrito': 'DISTRITO'}, inplace=True)
df_base.rename(columns={'Centro Poblado': 'CENTRO_POBLADO'}, inplace=True)
df_base.rename(columns={'Comunidad': 'COMUNIDAD'}, inplace=True)

df_base.rename(columns={'Ape. Paterno': 'PATERNO_JEFE_HOGAR'}, inplace=True)
df_base.rename(columns={'Ape. Materno': 'MATERNO_JEFE_HOGAR'}, inplace=True)
df_base.rename(columns={'Nombre': 'NOMBRES_JEFE_HOGAR'}, inplace=True)
df_base.rename(columns={'DNI': 'DNI_JEFE_HOGAR'}, inplace=True)
df_base.rename(columns={'Sexo': 'SEXO_JEFE_HOGAR'}, inplace=True)
df_base.rename(columns={'edad': 'EDAD_JEFE_HOGAR'}, inplace=True)
df_base.rename(columns={'Fecha de Nacimiento': 'FEC_NAC_JEFE_HOGAR'}, inplace=True)

df_base.rename(columns={'Número de Miembros': 'CANT'}, inplace=True)
df_base.rename(columns={'altitud': 'ALTURA'}, inplace=True)
df_base.rename(columns={'latitud': 'LATITUD'}, inplace=True)
df_base.rename(columns={'longitud': 'LONGITUD'}, inplace=True)

In [208]:
print(df06.shape[0])
print(df_base.shape[0])

20848
45439


In [209]:
df_base_AND_f06_inner = pd.merge(df_base, df06, on = 'CUV', suffixes={'_BASE', '_F06'})
df_base_AND_f06_left = pd.merge(df_base, df06, on = 'CUV', how = 'left', suffixes={'_BASE', '_F06'})
df_base_AND_f06_right = pd.merge(df_base, df06, on = 'CUV', how = 'right' ,suffixes={'_BASE', '_F06'})

print(df_base_AND_f06_inner.shape[0])
print(df_base_AND_f06_left.shape[0])
print(df_base_AND_f06_right.shape[0])

6692
45439
20848


<ipython-input-209-6acb27045d6d>:1: FutureWarning: Passing 'suffixes' as a <class 'set'>, is not supported and may give unexpected results. Provide 'suffixes' as a tuple instead. In the future a 'TypeError' will be raised.
  df_base_AND_f06_inner = pd.merge(df_base, df06, on = 'CUV', suffixes={'_BASE', '_F06'})
<ipython-input-209-6acb27045d6d>:2: FutureWarning: Passing 'suffixes' as a <class 'set'>, is not supported and may give unexpected results. Provide 'suffixes' as a tuple instead. In the future a 'TypeError' will be raised.
  df_base_AND_f06_left = pd.merge(df_base, df06, on = 'CUV', how = 'left', suffixes={'_BASE', '_F06'})
<ipython-input-209-6acb27045d6d>:3: FutureWarning: Passing 'suffixes' as a <class 'set'>, is not supported and may give unexpected results. Provide 'suffixes' as a tuple instead. In the future a 'TypeError' will be raised.
  df_base_AND_f06_right = pd.merge(df_base, df06, on = 'CUV', how = 'right' ,suffixes={'_BASE', '_F06'})


In [211]:
df_base_with_f06_outer = pd.merge(df_base, df06, on='CUV', how='outer', suffixes=('_BASE', '_F06'))


In [214]:
df_base_with_f06_outer.head(10)

,OBJECTID,CUV,CODIGO_CCPP_BASE,UBIGEO_BASE,DEPARTAMENTO_BASE,PROVINCIA_BASE,DISTRITO_BASE,CENTRO_POBLADO_BASE,COMUNIDAD_BASE,PATERNO_JEFE_HOGAR_BASE,...,CLASIFICACION_SOCIOECONOMICA_CSE,VIABILIDAD_TOTAL,VIABILIDAD_SOCIAL,VIABILIDAD_TECNICA,DNI_CYG,ACEPTA_ENTREVISTA,CRITERIO_SOCIAL,CRITERIO_TECNICO,FEC_EVALUACION,FECHA_CONSULTA_CSE
0,1.0,1,2108060037,210806.0,PUNO,MELGAR,NUÑOA,ACCOCUNCA,NaN,CHISLLA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.0,2,2108060037,210806.0,PUNO,MELGAR,NUÑOA,ACCOCUNCA,NaN,PACO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3.0,3,2108060037,210806.0,PUNO,MELGAR,NUÑOA,ACCOCUNCA,NaN,VILLANUEVA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4.0,4,2108060089,210806.0,PUNO,MELGAR,NUÑOA,CANGALLI PICHACANI,NaN,ARCE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5.0,5,2108060089,210806.0,PUNO,MELGAR,NUÑOA,CANGALLI PICHACANI,NaN,BARRAGAN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,6.0,6,2108060089,210806.0,PUNO,MELGAR,NUÑOA,CANGALLI PICHACANI,NaN,CHOQUEPATA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,7.0,7,2108060089,210806.0,PUNO,MELGAR,NUÑOA,CANGALLI PICHACANI,NaN,CHURA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,8.0,8,2108060089,210806.0,PUNO,MELGAR,NUÑOA,CANGALLI PICHACANI,NaN,CONDORI,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,9.0,9,2108060089,210806.0,PUNO,MELGAR,NUÑOA,CANGALLI PICHACANI,NaN,DIAZ,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,10.0,10,2108060089,210806.0,PUNO,MELGAR,NUÑOA,CANGALLI PICHACANI,NaN,FUENTES,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [216]:
df_base_with_f06_outer.shape[0]

59595

In [215]:
# Confirmamos que no hay duplicados en el resultado

# Counting duplicated values in the 'Name' column
duplicates_count = df_base_with_f06_outer.duplicated(subset='CUV').sum()
print("\nCount of duplicated values in the 'CUV' column:", duplicates_count)
# Count of duplicated values in the 'CUV' column: 5602


Count of duplicated values in the 'CUV' column: 0


In [212]:
# df_base_with_f06_outer.to_excel(file_temp_1)

In [262]:
df_new = df_base_with_f06_outer

In [263]:
constante_fuente_datos_ambos = 'FORMATO06, BASE_CONSOLIDADA'
constante_fuente_datos_FORMATO06 = 'FORMATO06'
constante_fuente_datos_base = 'BASE_CONSOLIDADA'


condition_1 = ( df_new['NUMERO'].notna() & df_new['OBJECTID'].notna())

condition_2 = ( df_new['OBJECTID'].isnull())
condition_3 = ( df_new['NUMERO'].isnull())

  # condition = (df['A'] == 2) | (df['B'] > 30)

# df_new['FUENTE_DATOS']=  np.where(condition_1, constante_fuente_datos_ambos, np.where(condition_2, constante_fuente_datos_FORMATO06, constante_fuente_datos_base) )


#   condition_1 = ( df_temp['ESTADO_DEDUCTIVO'].isnull() & (df_temp['VIABILIDAD_TOTAL'] == constante_estado_viabilidad_viable) )
#   condition_2 = ( df_temp['ESTADO_DEDUCTIVO'].isnull() & ~(df_temp['VIABILIDAD_TOTAL'] == constante_estado_viabilidad_viable) )

df_new.loc[condition_2, 'FUENTE_DATOS'] = constante_fuente_datos_FORMATO06
df_new.loc[condition_3, 'FUENTE_DATOS'] = constante_fuente_datos_base
df_new.loc[condition_1, 'FUENTE_DATOS'] = constante_fuente_datos_ambos

df_new.to_excel(file_temp_4)

  #   condition_1 = ( df_temp['ESTADO_DEDUCTIVO'].isnull() & (df_temp['VIABILIDAD_TOTAL'] == constante_estado_viabilidad_viable) )
  # condition_2 = ( df_temp['ESTADO_DEDUCTIVO'].isnull() & ~(df_temp['VIABILIDAD_TOTAL'] == constante_estado_viabilidad_viable) )

  # df_temp.loc[condition_1, 'ESTADO_DEDUCTIVO'] = constante_estado_no_deductivo
  # df_temp.loc[condition_2, 'ESTADO_DEDUCTIVO'] = constante_estado_sininfo



In [264]:
df_new['FUENTE_DATOS'].value_counts()

BASE_CONSOLIDADA               38747
FORMATO06                      14156
FORMATO06, BASE_CONSOLIDADA     6692
Name: FUENTE_DATOS, dtype: int64

In [267]:
# =============================================================================================
# POST : Crear los Estados
#================================================================================================
# reglas para llenar los campos faltantes, derivados de informacion existente
# Se llenan de esta forma, porque ya son DEDUCTIVOS, es decir han pasado por todas las etapas desde ser registrados, evaluados, y viables.
# viabilidad, considerando que todos los deductivos, previaente fueron REGISTRADOS, luego EVALUADOS, y luego viables, entonces se coloca el valor VIABLE , lara todos los registros

#===================================================
# Estado REGISTRO
#===================================================
df_new['ESTADO_REGISTRADO'] = constante_estado_registrado

condition_x = ((df_new['FUENTE_DATOS'] == constante_fuente_datos_FORMATO06) | (df_new['FUENTE_DATOS'] == constante_fuente_datos_FORMATO06))


#===================================================
# Estado de EVALUACION
#===================================================

# Sample conditions and constants
condition_1 = df_new['ACEPTA_ENTREVISTA'] == 'SI'
condition_2 = df_new['ACEPTA_ENTREVISTA'] == 'Rechazo de evaluación / Intervención'

conditions = [condition_1, condition_2]
values = [constante_estado_evaluacion_evaluado, constante_estado_evaluacion_rechaza]

# Default value when none of the conditions are True
default_value = constante_estado_evaluacion_otros

df_new['ESTADO_EVALUACION'] = np.select(conditions, values, default=default_value)



In [271]:
#===================================================
# Estado de VIABLE
#===================================================
condition_1 = df['VIABILIDAD_TOTAL'] == constante_estado_viabilidad_viable
condition_2 = df['VIABILIDAD_TOTAL'] == constante_estado_viabilidad_no_viable

conditions = [condition_1, condition_2]
values = [constante_estado_viabilidad_viable, constante_estado_viabilidad_no_viable]

# Default value when none of the conditions are True
default_value = constante_estado_sininfo

df_new['ESTADO_VIABLE'] = np.select(conditions, values, default=default_value)

ValueError: ignored

In [ ]:
df_new.info()

In [243]:


def crear_dataset_deductivos(file_input_deductivos, file_input_nucleo):

  df_deductivos = pd.read_excel(file_input_deductivos)
  df_nucleo = pd.read_excel(file_input_nucleo)

  print(df_nucleo.head(3))

  df_deductivos_nucleo = pd.merge(df_deductivos, df_nucleo,  on = 'CONVENIO', how = 'left')

  # ESTADO : CREAMOS EL ESTADO DEDUCTIVO PARA ESTA TABLA
  df_deductivos_nucleo['ESTADO_DEDUCTIVO'] = 'DEDUCTIVO'

  print(df_deductivos.shape[0])
  print(df_deductivos_nucleo.shape[0])
  # print(df_deductivos_nucleo.head())

  df_deductivos_nucleo.to_excel(file_output_3)
  # print(df_deductivos_nucleo.info())

  columns_selected_deductivos = ['ID', 'CUV','DNI','CODIGO_UGT','CONVENIO','NOMBRE','AP_PATERNO','DEDUCTIVO','CARTA','MOTIVO','HT','FEC_APROB_EXP_TECNICO','FEC_ACTA_INICIO_CONSTRUCCION','ESTADO_DEDUCTIVO']

  df_deductivos_nucleo = df_deductivos_nucleo[columns_selected_deductivos]
  # 130 registros

  return df_deductivos_nucleo

In [244]:

def union_df_with_deductivos(df, df_deductivos_nucleo):

  df_join = pd.merge(df, df_deductivos_nucleo, on = 'CUV')
  df_join_left = pd.merge(df, df_deductivos_nucleo, on = 'CUV', how = 'left', suffixes=('_F06', '_deductivo'))

  print(df_join_left['FEC_EVALUACION'])

  return df_join_left

In [255]:
##########################################################################3
# CALCULOS POST PROCESO
##########################################################################3

# Completar ESTADO DEDUCTIVO , luego del JOIN, en la nueva tabla totalizada

def add_and_transform_columns_with_deductivos(df):

  df_temp = df

  condition_1 = ( df_temp['ESTADO_DEDUCTIVO'].isnull() & (df_temp['VIABILIDAD_TOTAL'] == constante_estado_viabilidad_viable) )
  condition_2 = ( df_temp['ESTADO_DEDUCTIVO'].isnull() & ~(df_temp['VIABILIDAD_TOTAL'] == constante_estado_viabilidad_viable) )

  df_temp.loc[condition_1, 'ESTADO_DEDUCTIVO'] = constante_estado_no_deductivo
  df_temp.loc[condition_2, 'ESTADO_DEDUCTIVO'] = constante_estado_sininfo

  # FECHAS:
  # FEC_EVALUACION, FEC_APROB_EXP_TECNICO,FEC_ACTA_INICIO_CONSTRUCCION
  #

    # Convert date strings to datetime objects
  df_temp['FEC_EVALUACION'] = pd.to_datetime(df_temp['FEC_EVALUACION'])
  df_temp['FEC_APROB_EXP_TECNICO'] = pd.to_datetime(df_temp['FEC_APROB_EXP_TECNICO'])
  df_temp['FEC_ACTA_INICIO_CONSTRUCCION'] = pd.to_datetime(df_temp['FEC_ACTA_INICIO_CONSTRUCCION'])

  # Calculate the difference between the two dates





  df_temp['days_diff_exp_eval'] = (df_temp['FEC_APROB_EXP_TECNICO'] - df_temp['FEC_EVALUACION']).dt.days
  df_temp['days_diff_acta_eval'] = (df_temp['FEC_ACTA_INICIO_CONSTRUCCION'] - df_temp['FEC_EVALUACION']).dt.days
  df_temp['days_diff_acta_exp'] = (df_temp['FEC_ACTA_INICIO_CONSTRUCCION'] - df_temp['FEC_APROB_EXP_TECNICO']).dt.days

  print(df_temp.info())

  print(df_temp['days_diff_acta_exp'])

    ############################
  # fillna para las fechas : se completa con 0, las que son nulas.
  # 0, es un valor que en la realidad NO es valido, porque desde la EVALUACION hasta la REALIZACION del expediente, o
  # DESDE la APROBACION DEL EXPEDIENTE TECNICO hasta la CONSTRUCCION, el tiempo es mayor que 0
  ############################

  # List of columns to fill with zeros
  columns_to_fill = ['days_diff_exp_eval', 'days_diff_acta_eval', 'days_diff_acta_exp', 'CANT_F06', 'CANT_BASE']
  # Fill NaN values with 0 only in the specified columns
  df_temp[columns_to_fill] = df_temp[columns_to_fill].fillna(0)

  return df_temp

In [248]:
df_deductivos_nucleo = crear_dataset_deductivos(file_deductivos, file_input_nucleo)
df_join_left = union_df_with_deductivos(df_new,df_deductivos_nucleo)


                 CONVENIO   COD UGT FEC_APROB_EXP_TECNICO  \
0  013-2017-LOR/VMVU/PNVR  013-2017            2017-03-13   
1  006-2022-PUN/VMVU/PNVR  006-2022            2022-03-25   
2  007-2022-PUN/VMVU/PNVR  007-2022            2022-03-25   

  FEC_ACTA_INICIO_CONSTRUCCION     GRUPO  
0                   2023-06-19  GRUPO 4B  
1                   2023-07-17   GRUPO 2  
2                   2023-06-19   GRUPO 2  
1017
1017
0               NaN
1               NaN
2               NaN
3               NaN
4               NaN
            ...    
59592    18-11-2022
59593    18-11-2022
59594    19-11-2022
59595    19-11-2022
59596    19-11-2022
Name: FEC_EVALUACION, Length: 59597, dtype: object


In [254]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59595 entries, 0 to 59594
Data columns (total 53 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   OBJECTID                          45439 non-null  float64       
 1   CUV                               59595 non-null  object        
 2   CODIGO_CCPP_BASE                  45439 non-null  object        
 3   UBIGEO_BASE                       45439 non-null  float64       
 4   DEPARTAMENTO_BASE                 45439 non-null  object        
 5   PROVINCIA_BASE                    45439 non-null  object        
 6   DISTRITO_BASE                     45439 non-null  object        
 7   CENTRO_POBLADO_BASE               45439 non-null  object        
 8   COMUNIDAD_BASE                    25519 non-null  object        
 9   PATERNO_JEFE_HOGAR_BASE           45439 non-null  object        
 10  MATERNO_JEFE_HOGAR                45420 non-nu

In [256]:
df_join_left = add_and_transform_columns_with_deductivos(df_join_left)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59597 entries, 0 to 59596
Data columns (total 69 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   OBJECTID                          45441 non-null  float64       
 1   CUV                               59597 non-null  object        
 2   CODIGO_CCPP_BASE                  45441 non-null  object        
 3   UBIGEO_BASE                       45441 non-null  float64       
 4   DEPARTAMENTO_BASE                 45441 non-null  object        
 5   PROVINCIA_BASE                    45441 non-null  object        
 6   DISTRITO_BASE                     45441 non-null  object        
 7   CENTRO_POBLADO_BASE               45441 non-null  object        
 8   COMUNIDAD_BASE                    25519 non-null  object        
 9   PATERNO_JEFE_HOGAR_BASE           45441 non-null  object        
 10  MATERNO_JEFE_HOGAR                45422 non-nu

In [259]:
# ver que otros campos de deductivos, deben ser tomados para completar segun salida de analisis?_deductivos.xlsx

df_join_left.to_excel(file_output_analisis_2)